In [39]:
import open3d as o3d
import numpy as np
import copy
import trimesh
import threading
import time
from multiprocessing.pool import ThreadPool

%run "../Notebooks/ClusteringPlottingMeshing.ipynb"
%run "../Notebooks/Stitching_functions.ipynb"

Clustering, Plotting and Meshing loaded at 09:41:36


In [9]:
model_mesh = o3d.io.read_triangle_mesh("../data/dental.stl")
model_mesh.scale(1000, [0,0,0])
model_mesh.compute_vertex_normals()
source_original = o3d.geometry.PointCloud()
source_original.points = model_mesh.vertices
source_original.estimate_normals()

source_original.paint_uniform_color([0.2,0.2,0.8])
target_original = o3d.io.read_point_cloud("../data/st_cropped_2.ply")
target_original.estimate_normals()
target_original.paint_uniform_color([0.8,0.2,0.2])

#source_original.paint_uniform_color([0.2,0.2,0.8])
#target_original = o3d.io.read_point_cloud("../data/plyfolder/points0.ply")
#target_original.estimate_normals()
#target_original.paint_uniform_color([0.8,0.2,0.2])


#print(source_original, target_original)

PointCloud with 299291 points.

In [10]:
def draw_results(src, tar, trans):
    src_copy = copy.deepcopy(src)
    tar_copy = copy.deepcopy(tar)
    tar_copy.transform(trans)
    o3d.visualization.draw_geometries([src_copy,tar_copy], mesh_show_back_face=True)

In [11]:
#m, t, c = trimesh.registration.icp(np.asarray(source_down.points), np.asarray(target_down.points), threshold = 0.02, max_iterations = 20)

In [12]:
#result = o3d.geometry.PointCloud()
#result.points = o3d.cpu.pybind.utility.Vector3dVector(t)
#result.estimate_normals()
#result.paint_uniform_color([0.2,0.8,0.2])

In [13]:
#o3d.visualization.draw_geometries([target,result], mesh_show_back_face=True)

In [14]:
#voxel_size = 0.01
#source_down = source.voxel_down_sample(voxel_size)

In [15]:
source = copy.deepcopy(model_mesh)
source = copy.deepcopy(source_original)
target = copy.deepcopy(target_original)



transformation_setup = np.linalg.inv(np.array([
 [ -0.42848023,   0.89551476,  -0.12024146,  18.96549334],
 [ -0.72105517,  -0.41909148,  -0.55176243,  -4.08932589],
 [ -0.54450357,  -0.14971857,   0.82528796, -14.90926351],
 [  0.        ,   0.        ,   0.        ,   1.        ]]))

transformation_setup = np.linalg.inv(np.array([
     [ -0.42769944,   0.89569105,  -0.12169932,  19.20310461],
     [ -0.7399105,   -0.42424645,  -0.52206073,  -6.14917153],
     [ -0.51923563,  -0.13323848,   0.84418118,  -14.54139077],
     [  0.,           0.,           0.,           1.        ]]))

transformation_setup = np.linalg.inv(np.array([
     [ -0.42936695,   0.89689573,  -0.10593428,  18.08900141],
     [ -0.73091052,  -0.4139897 ,  -0.54257012,  -4.68199364],
     [ -0.53048452,  -0.15553319,   0.83330402, -14.77113242],
     [  0.        ,   0.        ,   0.        ,   1.        ]]))

target.transform(transformation_setup)



voxel_size = 0.5
#source_down = source.voxel_down_sample(voxel_size)
target_down = target.voxel_down_sample(voxel_size)

#print(source_down, target_down)

In [28]:
o3d.visualization.draw_geometries([source,target_down], mesh_show_back_face=True)

In [16]:
#crop model to reduce processing time
boundingbox = o3d.geometry.OrientedBoundingBox.create_from_points(target.points)
cropped_mesh = copy.deepcopy(model_mesh).crop(boundingbox) 

o3d.visualization.draw_geometries([cropped_mesh,target,boundingbox], mesh_show_back_face=True)

In [ ]:
o3d.visualization.draw_geometries([cropped_mesh,target,boundingbox], mesh_show_back_face=True)

In [ ]:
dists = np.asarray(source.compute_point_cloud_distance(target))
print("Hausdorff:", max(dists))
print("Median:", np.median(dists))
print("Mean:", np.mean(dists))
print("Mean Squared Error:", np.mean(dists*dists))

Hausdorff: 50.9394356497245
Median: 26.624638329827974
Mean: 24.168652337745673
Mean Squared Error: 741.6561698445337

In [14]:
signed_distance = trimesh.proximity.signed_distance(convert(cropped_mesh), np.asarray(target_down.points)[0:1])

In [42]:
def calcDists(points):
    return abs(trimesh.proximity.signed_distance(convert(cropped_mesh), points))

def test(x):
    print(x)
    
points = np.asarray(target_down.points)[:5]
dists = np.array([0]*len(points))

step = 2
threads = []
pool = ThreadPool(processes=int(len(points)/step+0.5))

inputs = []

for i in range(0,len(points),step):
    inputs.append(points[i:min(i+step, len(points))])
    
async_result = pool.apply_async(test, [1,2,3]) # tuple of args for foo

# do some other stuff in the main process

dists = async_result.get()  # get the return value from your function.
    

TypeError: test() takes 1 positional argument but 3 were given

In [41]:
inputs

[array([[-38.69819494, -15.47167101,  47.65106266],
        [-35.76778987,  -6.40825341,  47.37738359]]),
 array([[-38.87628708,  -8.40698904,  44.96959176],
        [-29.48627627, -17.46554106,  44.49760963]]),
 array([[-32.26621802, -23.00100305,  42.07330425]])]

In [ ]:
dists = abs(signed_distance)
dists

In [23]:
print("Hausdorff:", max(dists))
print("Median:", np.median(dists))
print("Mean:", np.mean(dists))
print("Mean Squared Error:", np.mean(dists*dists))

Hausdorff: 1.1874147075251684
Median: 0.19985846435285887
Mean: 0.26034363958237244
Mean Squared Error: 0.11813057758589474


In [ ]:
trimesh.registration.icp(targt_down, cropped_mesh, initial=array([[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.], [0., 0., 0., 1.]]), threshold=1e-05, max_iterations=20, **kwargs

In [93]:
m, c = trimesh.registration.mesh_other(convert(cropped_mesh), np.asarray(target_down.points), samples=500, scale=True, icp_first=10, icp_final=50)

In [94]:
m

array([[ 0.99954558,  0.01361179,  0.03772956, -1.38495148],
       [-0.01387064,  1.00023202,  0.00660988, -0.98361391],
       [-0.03763517, -0.00712771,  0.99961641, -1.2478627 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [108]:
c

8.89720391870747e-06

In [97]:
draw_results(target, cropped_mesh, m)

In [109]:
src_copy = copy.deepcopy(source)
tar_copy = copy.deepcopy(target)
src_copy.transform(m)
    
distVectors = trimesh.proximity.closest_point(convert(src_copy), np.asarray(tar_copy.points))
dists = np.array([abs(np.sqrt(sum(d*d))) for d in distVectors[0]])
    
print("Hausdorff:", max(dists))
print("Median:", np.median(dists))
print("Mean:", np.mean(dists))
print("Mean Squared Error:", np.mean(dists*dists))

Hausdorff: 67.57279519560882
Median: 58.59395106523382
Mean: 59.23389002152368
Mean Squared Error: 3517.7227142114502


In [111]:
src_copy = copy.deepcopy(source)
tar_copy = copy.deepcopy(target)
src_copy.transform(m)



o3d.visualization.draw_geometries([src_copy,tar_copy,boundingbox], mesh_show_back_face=True)

TypeError: (): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.geometry.PointCloud, arg0: open3d.cpu.pybind.utility.Vector3dVector) -> None

Invoked with: PointCloud with 299291 points., array([0.57070565, 0.55467526, 0.55534027, ..., 0.58351682, 0.58355203,
       0.58366006])